In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%%capture
!pip install pyannote.audio
!pip install pyannote.core
!pip install torch
!pip install torchaudio
!pip install faster-whisper
!pip install --no-build-isolation nemo_toolkit[asr]==1.22.0
#!pip install wget

In [12]:
#segment
from pyannote.audio import Audio
from pyannote.core import Segment
import torch, torchaudio
#import ffmpeg
import os
#import librosa

#embed
from faster_whisper import WhisperModel
#from segment import segment_audio
#from embed import embed_audio
import tensorflow as tf

#diarize
from nemo.collections.asr.models.msdd_models import NeuralDiarizer
import wget

# Segmentation

In [5]:
audio = Audio(mono='downmix')

def segmentation(trans_segment,
            waveform,
            #video,
            sr,
            seg_num,
            segment_audio=True,
            segment_video=False):

    if segment_audio:
        #segmenting
        start = trans_segment.start
        end = trans_segment.end
        clip = Segment(start, end)
        audio_segment, sr = audio.crop(waveform, clip) #as a tensor

        #extracting file name and extension
        #name, extension = os.path.splitext(waveform)
        #if not os.path.exists("%s_segments" % (name)):
         #   os.makedirs("%s_segments" % (name))

        #saving segment into file
        #torchaudio.save("./%s_segments/%s_seg_%x%s" % (name, name, seg_num, extension),
         #                audio_segment, sr, format="wav")

        return audio_segment

# Feature Extraction

In [6]:
def load_model():
    # load the pre-trained checkpoints
    model_checkpoint_loc = '/content/drive/MyDrive/master/data_internship/WavLM-Large.pt'
    checkpoint = torch.load(model_checkpoint_loc) #should be .pt file
    cfg = WavLMConfig(checkpoint['cfg'])
    model = WavLM(cfg)
    model.load_state_dict(checkpoint['model'])
    model.eval()
    return model, cfg

def embed_audio(model, cfg, audio_segment):
    #audio_segment = segment_audio(trans_segment, waveform, sr)
    if cfg.normalize:
        wav_input_16khz = torch.nn.functional.layer_norm(audio_segment, audio_segment.shape)
    rep = model.extract_features(wav_input_16khz)[0]
    return rep

In [25]:
import sys
sys.path.append('/content/drive/MyDrive/master/data_internship')
from WavLM import WavLM, WavLMConfig

In [26]:
#loading feature extraction model
model, cfg = load_model()

# Run

## Faster-whisper Transcription

In [ ]:
model_size = "large-v3"

# Run on GPU with FP16
#model = WhisperModel(model_size, device="cuda", compute_type="float16")

# or run on GPU with INT8
# model = WhisperModel(model_size, device="cuda", compute_type="int8_float16")
# or run on CPU with INT8
model = WhisperModel(model_size, device="cpu", compute_type="int8")

audioin = "/content/drive/MyDrive/master/data_internship/testaudio.wav"

segments, info = model.transcribe(audioin, beam_size=5)

print("Detected language '%s' with probability %f" % (info.language, info.language_probability))

## Segment and embed

In [ ]:
#--- PARAMS ---#
#sampling rate
sr = 16000
#segment number initialization
seg_num = 0

for segment in segments:
    #print(segment.text)
    #calling segmentation creates
    seg = segmentation(segment, audioin, sr, seg_num)
    seg_num+=1
    seg1= torch.randn(1,30000)
    emb = embed_audio(model, cfg, seg)
    print(emb)

tensor([[[-0.4782, -0.0683, -0.3483,  ..., -0.1665,  0.0502, -0.1553],
         [-0.3683, -0.1869, -0.4032,  ..., -0.1569,  0.0446, -0.1049],
         [-0.3760, -0.2513, -0.3373,  ..., -0.1399,  0.0418, -0.2199],
         ...,
         [-0.2296,  0.3407, -0.1926,  ..., -0.2464, -0.0885, -0.2905],
         [-0.2521,  0.3283, -0.0810,  ..., -0.2422, -0.0836, -0.2940],
         [-0.3297,  0.2685, -0.1152,  ..., -0.3087,  0.0198, -0.1060]]],
       grad_fn=<NativeLayerNormBackward0>)


#Diarization (in progress, don't run)

In [8]:
ROOT = os.getcwd()
temp_path = os.path.join(ROOT, "temp_outputs")

In [14]:
def create_config(output_dir):
    DOMAIN_TYPE = "general" #
    CONFIG_FILE_NAME = f"diar_infer_{DOMAIN_TYPE}.yaml"
    CONFIG_URL = f"https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/speaker_tasks/diarization/conf/inference/{CONFIG_FILE_NAME}"
    MODEL_CONFIG = os.path.join(output_dir, CONFIG_FILE_NAME)
    if not os.path.exists(MODEL_CONFIG):
        MODEL_CONFIG = wget.download(CONFIG_URL, output_dir)

In [ ]:
msdd_model = NeuralDiarizer(cfg=create_config(temp_path)).to("cuda")
msdd_model.diarize()

del msdd_model
torch.cuda.empty_cache()